In [1]:
import copy
import os
from tqdm import tqdm
import numpy as np
from math import floor

from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DataCollatorForCompletionOnlyLM
from peft import get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, prepare_model_for_kbit_training

from utils import *
from utils.utils import default_evaluation, save_dataset_test

from dataclasses import dataclass, field, asdict
from typing import Optional
from transformers import HfArgumentParser, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
import os
import json
from accelerate import Accelerator
import torch
from datetime import datetime, timedelta

from trl import SFTTrainer
from transformers import TrainerCallback

/home/gabriel.talasso/FT_LLM_FL/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
dataset_name = 'multitask'
model_name = 'HuggingFaceTB/SmolLM-135M'

In [4]:
dataset, dataset_test = get_dataset(dataset_name, None, 0.8)
dataset =      process_sft_dataset(dataset_name, dataset,      400000)
dataset_test = process_sft_dataset(dataset_name, dataset_test, 400000)

Using the latest cached version of the module from /home/gabriel.talasso/.cache/huggingface/modules/datasets_modules/datasets/Samsung--samsum/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e (last modified on Tue Mar 11 13:33:01 2025) since it couldn't be found locally at Samsung/samsum, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/gabriel.talasso/.cache/huggingface/modules/datasets_modules/datasets/Harvard--gigaword/cd40ed63b2731b6da1f321fba3f70998c2d7de54730f50e0c20f6ac37f323a40 (last modified on Tue Mar 11 13:37:41 2025) since it couldn't be found locally at Harvard/gigaword, or remotely on the Hugging Face Hub.


Dataset Splited into TRAIN: 71668 and TEST: 17917


In [5]:
local_datasets = []
tasks = ['boolq', 'gigaword', 'samsum', 'webnlg']
n_clients_in_cluster = 4 // len(tasks)

for i in range(4):
    task = tasks[i // n_clients_in_cluster]
    cluster_dataset = dataset.filter(lambda x: x['task'] == task)
    cluster_dataset = cluster_dataset.shuffle(seed=0)

    local_datasets.append(cluster_dataset.shard(n_clients_in_cluster, i % n_clients_in_cluster))

In [6]:
EVAL_SIZE = 1000

In [7]:
local_datasets_test = []
for i in range(4):
    task = tasks[i // n_clients_in_cluster]
    cluster_dataset_test = dataset_test.filter(lambda x: x['task'] == task)
    cluster_dataset_test = cluster_dataset_test.shuffle(seed=0)

    aux = cluster_dataset_test.shard(n_clients_in_cluster, i % n_clients_in_cluster)
    aux = aux.select(range(EVAL_SIZE))  # Limit to 1000 samples for testing
    local_datasets_test.append(aux)


In [8]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
# Copy the model to each device
device_map = {"": Accelerator().local_process_index}
torch_dtype = torch.bfloat16

In [9]:
peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

In [10]:
def get_model_and_tokenizer():

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map=device_map,
        trust_remote_code=True,
        torch_dtype=torch_dtype,
    )
    print(f"Model loaded from {model_name}")


    model = prepare_model_for_kbit_training(
                model, use_gradient_checkpointing=True
            )

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

    model.enable_input_require_grads()

    # ===== Define the global and local models =====
    global_dict = copy.deepcopy(get_peft_model_state_dict(model))
    local_dict_list = [copy.deepcopy(global_dict) for i in range(4)]

    # ===== Define the tokenizer =====
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, padding_side="right")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.unk_token   # following vicuna

    if tokenizer.eos_token == tokenizer.unk_token or tokenizer.pad_token == tokenizer.eos_token:
        tokenizer.add_special_tokens({'pad_token': '<pad>'})
        print(f"Pad token is set to {tokenizer.pad_token}.")

    print('Special tokens:', tokenizer.special_tokens_map)
    model.resize_token_embeddings(len(tokenizer))

    return model, tokenizer

In [11]:
max_steps=50
num_train_epochs=1
num_rounds=10
eval_round="10,25,50,100"
batch_size=8
batch_size_eval=128
gradient_accumulation_steps=1
seq_length=1024
num_clients=8
sample_clients=8
lora_r=8
lora_alpha=16  # twice of lora_r
lr=5e-4

In [12]:
output_dir = 'output_centralized_tests50'
eval_batch_size = 64

In [13]:
training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=lr,
        logging_steps=100,
        num_train_epochs=num_train_epochs,
        max_steps=max_steps,
        report_to=None,
        save_steps=1000,
        save_total_limit=10,
        push_to_hub=False,
        hub_model_id=None,
        gradient_checkpointing=True,
        lr_scheduler_type="constant",
        #max_seq_length=script_args.seq_length,
    )

In [14]:
_,  tokenizer = get_model_and_tokenizer()
formatting_prompts_func, response_template = get_formatting_prompts_func('alpaca', tokenizer.eos_token)
if response_template:
    response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[2:]   # Now we have it like in the dataset texts: `[2277, 29937, 4007, 22137, 29901]` for Llama2
    data_collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)
    packing = False
else:
    data_collator = None
    packing = True

Model loaded from HuggingFaceTB/SmolLM-135M
trainable params: 460,800 || all params: 134,975,808 || trainable%: 0.3414


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Pad token is set to <pad>.
Special tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<|endoftext|>', '<|im_start|>', '<|im_end|>', '<repo_name>', '<reponame>', '<file_sep>', '<filename>', '<gh_stars>', '<issue_start>', '<issue_comment>', '<issue_closed>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<jupyter_script>', '<empty_output>']}


In [15]:
def default_evaluation(model, tokenizer, dataset, client_id, round, formatting_prompts_func, script_args, cluster_id=None):
    """
    Default evaluation function to compute model responses and ROUGE scores.
    """
    print("Evaluating model...")

    model.config.use_cache = True
    # Apply template to dataset
    dataset = utils.apply_template_to_dataset(dataset, formatting_prompts_func)
    dataset_length = len(dataset)
    eval_responses = utils.get_model_responses(model, tokenizer, dataset, batch_size=eval_batch_size)
    dataset_with_responses = dataset.select(range(len(dataset)))
    dataset_with_responses = dataset_with_responses.add_column('model_responses', eval_responses)
    scores = utils.calcule_rogue1(eval_responses, dataset_with_responses)
    print(f"Evaluation scores: {scores}")

    #verify output directory
    if not os.path.exists(os.path.join(output_dir, "evals")):
        os.makedirs(os.path.join(output_dir, "evals"))
    # Save evaluation results
    with open(os.path.join(output_dir, f"evals/rouge_client_{client_id}_cluster_{cluster_id}_round_{round}.json"), 'w') as f:
        scores['dataset_length'] = dataset_length
        json.dump(scores, f, indent=4)

    model.config.use_cache = False

In [16]:
for i, local_dataset in enumerate(local_datasets):

    model, tokenizer = get_model_and_tokenizer()

    test_dataset = local_datasets_test[i]

    print(f"Evaluating Pretrained Model on {test_dataset} for task {i}")
    default_evaluation(
                    model=model,
                    tokenizer=tokenizer,
                    dataset=test_dataset,
                    client_id=i,
                    round=0, #with respect to model from the previous round
                    formatting_prompts_func=formatting_prompts_func,
                    script_args=None,
                    cluster_id=0,
                )

    trainer = SFTTrainer(
            model=model,
            processing_class=tokenizer,
            args=training_args,
            #max_seq_length=script_args.seq_length,
            train_dataset=local_dataset,
            formatting_func=formatting_prompts_func,
            data_collator=data_collator,
            #packing=packing,
            #dataset_text_field=dataset_text_field,
        )

    print('Training model...')
    trainer.train()

    print('Saving model...')
    model.save_pretrained(os.path.join(output_dir, f"model_client_{i}_cluster_0_round_0"))

    print('Eval_posting model...')
    default_evaluation(
                    model=model,
                    tokenizer=tokenizer,
                    dataset=test_dataset,
                    client_id=i,
                    round=1, 
                    formatting_prompts_func=formatting_prompts_func,
                    script_args=None,
                    cluster_id=0,
                )
    
    print('Cleaning up...')
    del model
    del tokenizer
    torch.cuda.empty_cache()
    print('Done with client', i)
    

Model loaded from HuggingFaceTB/SmolLM-135M
trainable params: 460,800 || all params: 134,975,808 || trainable%: 0.3414
Pad token is set to <pad>.
Special tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<|endoftext|>', '<|im_start|>', '<|im_end|>', '<repo_name>', '<reponame>', '<file_sep>', '<filename>', '<gh_stars>', '<issue_start>', '<issue_comment>', '<issue_closed>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<jupyter_script>', '<empty_output>']}
Evaluating Pretrained Model on Dataset({
    features: ['question', 'answer', 'passage', 'instruction', 'response', 'task', 'gem_id', 'gem_parent_id', 'input', 'target', 'references', 'category', 'webnlg_id', 'id', 'dialogue', 'summary', 'document'],
    num_rows: 1000
}) for task 0
Evaluating model...


  0%|          | 0/16 [00:00<?, ?it/s]

Generating responses for batch 1/16


  6%|▋         | 1/16 [00:46<11:32, 46.17s/it]

Generating responses for batch 2/16


...

Generating responses for batch 15/16


 94%|█████████▍| 15/16 [09:51<00:39, 39.84s/it]

Generating responses for batch 16/16


100%|██████████| 16/16 [10:29<00:00, 39.34s/it]


Evaluation scores: {'rouge1': np.float64(0.4289135046595898), 'rouge2': np.float64(0.41690350201907306), 'rougeL': np.float64(0.4287601031867354), 'rougeLsum': np.float64(0.4294298872694082)}
Cleaning up...
Done with client 3


In [17]:
#Mixturing all datasets

dataset.shuffle(seed=0)
dataset = dataset.select(range(10000))  # Limit to 400000 samples for training

model, tokenizer = get_model_and_tokenizer()

trainer = SFTTrainer(
        model=model,
        processing_class=tokenizer,
        args=training_args,
        #max_seq_length=script_args.seq_length,
        train_dataset=dataset,
        formatting_func=formatting_prompts_func,
        data_collator=data_collator,
        #packing=packing,
        #dataset_text_field=dataset_text_field,
    )

print('Training model...')
trainer.train()

print('Saving model...')
model.save_pretrained(os.path.join(output_dir, f"model_client_all_cluster_0_round_0"))

for i, test_dataset in enumerate(local_datasets_test):

    print('Eval_posting model...')
    default_evaluation(
                    model=model,
                    tokenizer=tokenizer,
                    dataset=test_dataset,
                    client_id='all',
                    round=1, 
                    formatting_prompts_func=formatting_prompts_func,
                    script_args=None,
                    cluster_id=i,
                )

print('Cleaning up...')
del model
del tokenizer
torch.cuda.empty_cache()
print('Done with client', i)

Model loaded from HuggingFaceTB/SmolLM-135M
trainable params: 460,800 || all params: 134,975,808 || trainable%: 0.3414
Pad token is set to <pad>.
Special tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<|endoftext|>', '<|im_start|>', '<|im_end|>', '<repo_name>', '<reponame>', '<file_sep>', '<filename>', '<gh_stars>', '<issue_start>', '<issue_comment>', '<issue_closed>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<jupyter_script>', '<empty_output>']}


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Training model...


Step,Training Loss


/home/gabriel.talasso/FT_LLM_FL/.venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


...

Generating responses for batch 15/16


 94%|█████████▍| 15/16 [09:56<00:39, 39.96s/it]

Generating responses for batch 16/16


100%|██████████| 16/16 [10:34<00:00, 39.64s/it]


Evaluation scores: {'rouge1': np.float64(0.2978158291546483), 'rouge2': np.float64(0.29063551267582544), 'rougeL': np.float64(0.2973015274532096), 'rougeLsum': np.float64(0.2976449026477013)}
Cleaning up...
Done with client 3
